In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator
from h2o.transforms.decomposition import H2OPCA
import tempfile

In [2]:
h2o.init(max_mem_size="198G", nthreads = 30)
#nthreads = 30

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /opt/tljh/user/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpbibqjmow
  JVM stdout: /tmp/tmpbibqjmow/h2o_jupyter_chandler_vaughn_started_from_python.out
  JVM stderr: /tmp/tmpbibqjmow/h2o_jupyter_chandler_vaughn_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_jupyter_chandler_vaughn_rntgds
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,198 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,30
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
# create an export checkpoints directory
checkpoints_dir = tempfile.mkdtemp()

dataset = h2o.import_file('final_dataset_new.csv')
dataset.shape

Parse progress: |█████████████████████████████████████████████████████████| 100%


(59977, 8090)

In [4]:
# Identify predictors and response
x = dataset.columns
y = "genre"
x.remove(y)

#dataset[y].unique()
dataset[y] = dataset[y].asfactor()

In [5]:
#impute any na's to zero
dataset[x].impute(column = 0, values = [0 for c in range(dataset[x].ncol)])
dataset[x].isna().any()

False

In [6]:
#subset the data so that we can work with it
#eventually we need to do all data for modeling
#using splitframe so we get random rows
#subset_dataset_train, subset_dataset_test, therest = dataset.split_frame(ratios=[0.04, 0.04])

In [7]:
print("Rows To Process: " + str(dataset.nrows))
print("Dataframe Shape: " + str(dataset.shape))

Rows To Process: 59977
Dataframe Shape: (59977, 8090)


In [12]:
#subset the data so that we can work with it
#eventually we need to do all data for modeling
#subset_dataset = dataset[range(0,2000,1),:]
dataset[y] = dataset[y].asfactor()
dataset[x] = dataset[x].asnumeric() 


# Setup with the Power pca_method
pca_decomp = H2OPrincipalComponentAnalysisEstimator(k = 200, 
                                                    max_iterations = 1000000 , 
                                                    transform = "Standardize", 
                                                    pca_method = "Power",
                                                    use_all_factor_levels = True, 
                                                    impute_missing = True,
                                                    compute_metrics = True,
                                                    export_checkpoints_dir = checkpoints_dir)

In [ ]:
# Train PCA
pca_decomp.train(x=dataset.names, 
                 training_frame = dataset)

pca Model Build progress: |███████████████████████████████████████████████| 100%

In [1]:
# View the importance of components
var_imp = pca_decomp.varimp(use_pandas=True)
var_imp

NameError: name 'pca_decomp' is not defined

In [ ]:
import matplotlib.pyplot as plt
scree = pd.DataFrame(var_imp).T[1]
scree.reset_index()
scree = scree[1:] #take the data less the header row
scree = pd.DataFrame(scree)
print('Proportion of Variance Explained "Scree Plot"')
scree.plot()

In [ ]:
import matplotlib.pyplot as plt
c = pd.DataFrame(var_imp).T[2]
c.reset_index()
c = c[1:] #take the data less the header row
c = pd.DataFrame(c)
print('Cumulative Proportion of Variance Explained')
c.plot()

In [ ]:
# View the eigenvectors
pca_decomp.rotation()

# See the whole table with table.as_data_frame()

In [ ]:
dataset_new = pca_decomp.predict(dataset)

In [ ]:
dataset_new

In [ ]:
#append the labels for a new dataset
dataset_new = dataset_new.cbind(dataset[y])
h2o.export_file(subset_dataset_new, 'final_dataset_full_PCA.csv')

In [10]:
h2o.cluster().shutdown()

H2O session _sid_abfd was not closed properly.
